In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
#load data
lottery_data = pd.read_csv('NumerosJugadosPanama.csv')
lottery_data.head()

,Tipo de sorteo,Fecha,Primer Premio,Letras,Serie,Folio,Segundo Premio,Tercer Premio
0,Dominical,1/10/2023,9076,CBAD,21,8,8719,2534
1,Intermedio,4/10/2023,9066,ACAB,4,11,3720,9941
2,Dominical,9/10/2023,1600,CAAC,22,11,348,4120
3,Dominical,15-10-2023,6012,BACC,25,15,1099,2766
4,Dominical,22-10-2023,7691,CDCC,22,13,7485,308


In [3]:
# Fixing the lottery numbers to ensure they are in a four-digit format
lottery_data['Primer Premio'] = lottery_data['Primer Premio'].apply(lambda x: f'{x:04d}')
lottery_data['Segundo Premio'] = lottery_data['Segundo Premio'].apply(lambda x: f'{x:04d}')
lottery_data['Tercer Premio'] = lottery_data['Tercer Premio'].apply(lambda x: f'{x:04d}')

# Standardizing the date format
lottery_data['Fecha'] = pd.to_datetime(lottery_data['Fecha'], dayfirst=True).dt.strftime('%Y-%m-%d')

# Display the modified dataframe to verify changes
lottery_data.head()

,Tipo de sorteo,Fecha,Primer Premio,Letras,Serie,Folio,Segundo Premio,Tercer Premio
0,Dominical,2023-10-01,9076,CBAD,21,8,8719,2534
1,Intermedio,2023-10-04,9066,ACAB,4,11,3720,9941
2,Dominical,2023-10-09,1600,CAAC,22,11,0348,4120
3,Dominical,2023-10-15,6012,BACC,25,15,1099,2766
4,Dominical,2023-10-22,7691,CDCC,22,13,7485,0308


In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import pandas as pd
import numpy as np

# Assuming lottery_data is your DataFrame

# One-Hot Encoding for 'Tipo de sorteo'
one_hot_encoder = OneHotEncoder(sparse=False)
X = one_hot_encoder.fit_transform(lottery_data[['Tipo de sorteo']])

# Normalizing the lottery numbers (you might consider different approaches here)
scaler = MinMaxScaler()
y = scaler.fit_transform(lottery_data[['Primer Premio']])

# Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Make predictions
predictions = model.predict(X_test)

Epoch 1/10
8/8 [==============================] - 2s 40ms/step - loss: 0.0065 - val_loss: 0.0032
Epoch 2/10
8/8 [==============================] - 0s 10ms/step - loss: 0.0053 - val_loss: 0.0030
Epoch 3/10
8/8 [==============================] - 0s 14ms/step - loss: 0.0048 - val_loss: 0.0028
Epoch 4/10
8/8 [==============================] - 0s 14ms/step - loss: 0.0046 - val_loss: 0.0029
Epoch 5/10
8/8 [==============================] - 0s 17ms/step - loss: 0.0046 - val_loss: 0.0028
Epoch 6/10
8/8 [==============================] - 0s 15ms/step - loss: 0.0047 - val_loss: 0.0030
Epoch 7/10
8/8 [==============================] - 0s 13ms/step - loss: 0.0047 - val_loss: 0.0027
Epoch 8/10
8/8 [==============================] - 0s 11ms/step - loss: 0.0046 - val_loss: 0.0027
Epoch 9/10
8/8 [==============================] - 0s 11ms/step - loss: 0.0047 - val_loss: 0.0028
Epoch 10/10
3/3 [==============================] - 0s 3ms/step


In [5]:
test_loss = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')

3/3 [==============================] - 0s 7ms/step - loss: 0.0053
Test loss: 0.0053090923465788364


In [6]:
# Making predictions
# Let's say you want to make predictions for each type of draw
draw_types = ['Dominical', 'Intermedio', 'Zodiaco']  # Add all types of draws here
encoded_draw_types = one_hot_encoder.transform(np.array(draw_types).reshape(-1, 1))

# Use the model to make predictions
predictions = model.predict(encoded_draw_types)

# Inverse transform the predictions to get actual lottery numbers
predicted_numbers = scaler.inverse_transform(predictions)

# Displaying predictions
for draw_type, predicted_number in zip(draw_types, predicted_numbers):
    print(f"Predicted {draw_type}: {predicted_number[0]:.0f}")

1/1 [==============================] - 0s 42ms/step
Predicted Dominical: 5150
Predicted Intermedio: 4561
Predicted Zodiaco: 4356


c:\NLP_Heat\Py\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
